### torch.autograd를 사용한 자동 미분

In [2]:
import torch

x = torch.ones(5) # input
y = torch.zeros(3) # label
w = torch.randn(5, 3, requires_grad=True) # 텐서 기울기 계산 O >> backpropagation 수행
b = torch.randn(3, requires_grad=True) # bias
z = torch.matmul(x,w)+b # 행렬곱:(n,b)*(b*n)=(n*n)
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

# 이때 w,b는 최적화해야하는 parameter이므로 해당 텐서에 requires_grad=True로 설정해 변수에 대한 loss function의 변화도를 계산할 수 있도록 한다.

In [3]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x0000016B532F69B0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000016B5BC725F0>


### 변화도(Gradient 계산)
- w,b 최적화를 위해서는 loss 함수의 derivative(도함수)가 필요
- loss.bachward() 호출 후 w.grad와 b.grad 가져옴


In [4]:
loss.backward() # 역전파를 수행하고 w,b를 업데이트 
print(w.grad)
print(b.grad)


tensor([[0.3060, 0.0045, 0.0974],
        [0.3060, 0.0045, 0.0974],
        [0.3060, 0.0045, 0.0974],
        [0.3060, 0.0045, 0.0974],
        [0.3060, 0.0045, 0.0974]])
tensor([0.3060, 0.0045, 0.0974])


In [6]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

# 순전파 연산만 필요한 경우 방법 1
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

# 방법2
z = torch.matmul(x, w)+b
z_det = z.detach() # z의 경사 계산 멈춤
print(z_det.requires_grad)

True
False
False


대부분의 경우, 스칼라 손실 함수를 가지고 일부 매개변수와 관련한 변화도를 계산해야 합니다. 그러나 출력 함수가 임의의 텐서인 경우가 있습니다. 이럴 때, PyTorch는 실제 변화도가 아닌 야코비안 곱(Jacobian product)을 계산

In [7]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True) # 야코비안 곱 계산
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)# 야코비안 곱 계산
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)# 야코비안 곱 계산
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
